## Projects:

- Diffusion Models;
- YOLO versions  - object detection;
- Segment Anything Model (https://segment-anything.com/) (Liked);
- Mathematical Integration;
- BERT fine-tuning - Text classification (compare with logistic regression and TF-IDF vectors);
- CNN visualization - review the different techniques used to analyze CNN predictions;
- Monte Carlo dropout - review briefly the concept of uncertainty and what are the main techniques used to deal with uncertainty in deep learning;
- Deep vs Wide Neural Networks - Try to understand the effect of adding layers or adding parameters in a layer;
- Overparameterization regime;
- Neural ODEs - Neural Networks and ordinary differential equations are the two main modelization techniques used nowadays, review papers explaining some evolution of the idea of Neural ODE.

General observations:

- Presentation of 15 min 
- 70% grade
- No need to submit the code

Evaluation aspects:

- Clarity
- Understanding of concepts
- Depth of research and/or experiments

In [3]:
import torch
import numpy as np
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
from segment_anything import SamPredictor, sam_model_registry
from sklearn.model_selection import train_test_split
from tqdm import tqdm


# Load SAM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam = sam_model_registry["vit_h"](checkpoint="C:/Users/elisa/OneDrive/Desktop/DL_Final/sam_vit_h_4b8939.pth").to(device)
predictor = SamPredictor(sam)
print("✅ SAM Model loaded successfully!")

# Path to dataset
dataset_path = "C:/Users/elisa/OneDrive/Desktop/DL_Final/Breast_Images/"


✅ SAM Model loaded successfully!


In [4]:
# Path to dataset
dataset_path = "C:/Users/elisa/OneDrive/Desktop/DL_Final/Breast_Images/"

# Define the class mapping
class_map = {"normal": 0, "benign": 1, "malignant": 2}

# List to store dataset entries
data = []

# Loop through each category
for category in ["normal", "benign", "malignant"]:
    category_path = os.path.join(dataset_path, category)
    
    # Get all image files in the category folder
    for file_name in os.listdir(category_path):
        if file_name.endswith(('.png', '.jpg', '.jpeg')):  # Check if it's an image
            image_path = os.path.join(category_path, file_name)
            data.append({"image_path": image_path, "label": class_map[category]})

# Convert to Pandas DataFrame
df = pd.DataFrame(data)

In [5]:
df

,image_path,label
0,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,0
1,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,0
2,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,0
3,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,0
4,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,0
...,...,...
1350,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,2
1351,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,2
1352,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,2
1353,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,2


In [6]:
df.shape

(1355, 2)

We had to manually remove some images from the dataset as they contained human input identifying where the tumor is. In this way, we avoided training the CNN with image modified manually to indicate where the tumor is. 
From 1578 now we have 1357, so we lost 14% of our dataset.

The code below takes the ground thruth information created by the doctor and draw a bouding box around the actual breast ultrasound (adding some pixels to consdier a bigger area). It then crops the image around the bounding box creating ad hoc images of the turmor area. 
The code takes into account the presence of multiple masks for each image creating multiple cropped images for the same tumor image.
We created random bounding box to create random images of normal ultrasound to train our CNN to recognize when there is no tumor.

In [10]:
# Define output directories for cropped tumors
output_dir = "Cropped_Tumors"
benign_dir = os.path.join(output_dir, "benign")
malignant_dir = os.path.join(output_dir, "malignant")
normal_dir = os.path.join(output_dir, "normal")

os.makedirs(benign_dir, exist_ok=True)
os.makedirs(malignant_dir, exist_ok=True)
os.makedirs(normal_dir, exist_ok=True)

# Create a dictionary to store multiple masks for each image
image_mask_pairs = {}

# Store last bounding box from benign/malignant images
last_bbox = None  

# Iterate through the DataFrame to pair images and masks
for index, row in df.iterrows():
    image_path = row["image_path"]
    
    if "_mask" in image_path:  # This is a mask
        # Extract the base image path without any mask suffix
        base_image_path = (
            image_path.replace("_mask_2.png", ".png")
                      .replace("_mask_1.png", ".png")
                      .replace("_mask.png", ".png")
        )
        
        if base_image_path not in image_mask_pairs:
            image_mask_pairs[base_image_path] = []
        
        # Store all available masks in a list
        image_mask_pairs[base_image_path].append(image_path)

# Process benign and malignant images first
for index, row in tqdm(df.iterrows(), total=len(df)):
    image_path = row["image_path"]
    label = row["label"]  # 1 = benign, 2 = malignant

    # Skip normal images (they will be processed later)
    if label == 0 or "_mask" in image_path:
        continue  

    # Find corresponding masks (if any)
    mask_paths = sorted(image_mask_pairs.get(image_path, []))  

    # Load the ultrasound image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not load {image_path}")
        continue
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (1024, 1024))  # Resize for consistency

    # Process each mask for the image
    for mask_index, mask_path in enumerate(mask_paths):
        # Load the mask image
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"Warning: Could not load {mask_path}")
            continue
        mask = cv2.resize(mask, (1024, 1024))  # Ensure same size as image

        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) == 0:
            print(f"No tumor detected in {mask_path}")
            continue

        # Select the largest contour (assuming it's the tumor)
        largest_contour = max(contours, key=cv2.contourArea)

        # Get bounding box around the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)
        last_bbox = (x, y, w, h)  # Store last bounding box

        # Extract the tumor region
        tumor_crop = image[y:y+h, x:x+w]

        # Select folder based on label
        label_dir = benign_dir if label == 1 else malignant_dir

        # Save the cropped tumor image with mask index
        tumor_filename = os.path.join(label_dir, f"tumor_{index}_mask{mask_index}.png")
        cv2.imwrite(tumor_filename, cv2.cvtColor(tumor_crop, cv2.COLOR_RGB2BGR))

        print(f"Saved: {tumor_filename}")

# Now process only normal ultrasound images using the last known bounding box
for index, row in tqdm(df.iterrows(), total=len(df)):
    image_path = row["image_path"]
    label = row["label"]  # 0 = normal

    # Process only normal ultrasound images (skip masks)
    if label != 0 or "_mask" in image_path:
        continue  

    # Load the ultrasound image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not load {image_path}")
        continue
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (1024, 1024))  # Resize for consistency

    # If no bounding box has been recorded, skip the normal image
    if last_bbox is None:
        print(f"Skipping {image_path}: No bounding box available from previous tumors.")
        continue

    # Crop using the last bounding box
    x, y, w, h = last_bbox
    normal_crop = image[y:y+h, x:x+w]

    # Save the cropped normal image
    normal_filename = os.path.join(normal_dir, f"normal_{index}.png")
    cv2.imwrite(normal_filename, cv2.cvtColor(normal_crop, cv2.COLOR_RGB2BGR))

    print(f"Saved: {normal_filename}")

print("Processing complete. Cropped tumor images and normal images are saved in:", output_dir)


 21%|██        | 278/1355 [00:00<00:00, 2530.57it/s]

Saved: Cropped_Tumors\benign\tumor_266_mask0.png
Saved: Cropped_Tumors\benign\tumor_268_mask0.png
Saved: Cropped_Tumors\benign\tumor_270_mask0.png
Saved: Cropped_Tumors\benign\tumor_270_mask1.png
Saved: Cropped_Tumors\benign\tumor_273_mask0.png
Saved: Cropped_Tumors\benign\tumor_275_mask0.png
Saved: Cropped_Tumors\benign\tumor_277_mask0.png
Saved: Cropped_Tumors\benign\tumor_279_mask0.png
Saved: Cropped_Tumors\benign\tumor_281_mask0.png
Saved: Cropped_Tumors\benign\tumor_283_mask0.png
Saved: Cropped_Tumors\benign\tumor_285_mask0.png
Saved: Cropped_Tumors\benign\tumor_287_mask0.png
Saved: Cropped_Tumors\benign\tumor_289_mask0.png
Saved: Cropped_Tumors\benign\tumor_291_mask0.png
Saved: Cropped_Tumors\benign\tumor_293_mask0.png
Saved: Cropped_Tumors\benign\tumor_295_mask0.png
Saved: Cropped_Tumors\benign\tumor_297_mask0.png
Saved: Cropped_Tumors\benign\tumor_299_mask0.png
Saved: Cropped_Tumors\benign\tumor_301_mask0.png
Saved: Cropped_Tumors\benign\tumor_303_mask0.png
Saved: Cropped_Tumor

 39%|███▉      | 532/1355 [00:02<00:03, 216.11it/s] 

Saved: Cropped_Tumors\benign\tumor_506_mask0.png
Saved: Cropped_Tumors\benign\tumor_508_mask0.png
Saved: Cropped_Tumors\benign\tumor_510_mask0.png
Saved: Cropped_Tumors\benign\tumor_512_mask0.png
Saved: Cropped_Tumors\benign\tumor_514_mask0.png
Saved: Cropped_Tumors\benign\tumor_516_mask0.png
Saved: Cropped_Tumors\benign\tumor_518_mask0.png
Saved: Cropped_Tumors\benign\tumor_520_mask0.png
Saved: Cropped_Tumors\benign\tumor_522_mask0.png
Saved: Cropped_Tumors\benign\tumor_524_mask0.png
Saved: Cropped_Tumors\benign\tumor_526_mask0.png
Saved: Cropped_Tumors\benign\tumor_526_mask1.png
Saved: Cropped_Tumors\benign\tumor_529_mask0.png
Saved: Cropped_Tumors\benign\tumor_531_mask0.png
Saved: Cropped_Tumors\benign\tumor_533_mask0.png
Saved: Cropped_Tumors\benign\tumor_535_mask0.png
Saved: Cropped_Tumors\benign\tumor_537_mask0.png
Saved: Cropped_Tumors\benign\tumor_539_mask0.png
Saved: Cropped_Tumors\benign\tumor_541_mask0.png
Saved: Cropped_Tumors\benign\tumor_543_mask0.png
Saved: Cropped_Tumor

 48%|████▊     | 644/1355 [00:02<00:03, 190.06it/s]

...

Saved: Cropped_Tumors\normal\normal_226.png
Saved: Cropped_Tumors\normal\normal_228.png
Saved: Cropped_Tumors\normal\normal_230.png
Saved: Cropped_Tumors\normal\normal_232.png
Saved: Cropped_Tumors\normal\normal_234.png
Saved: Cropped_Tumors\normal\normal_236.png
Saved: Cropped_Tumors\normal\normal_238.png
Saved: Cropped_Tumors\normal\normal_240.png
Saved: Cropped_Tumors\normal\normal_242.png
Saved: Cropped_Tumors\normal\normal_244.png


 19%|█▊        | 253/1355 [00:01<00:06, 177.39it/s]

Saved: Cropped_Tumors\normal\normal_246.png
Saved: Cropped_Tumors\normal\normal_248.png
Saved: Cropped_Tumors\normal\normal_250.png
Saved: Cropped_Tumors\normal\normal_252.png
Saved: Cropped_Tumors\normal\normal_254.png
Saved: Cropped_Tumors\normal\normal_256.png
Saved: Cropped_Tumors\normal\normal_258.png
Saved: Cropped_Tumors\normal\normal_260.png
Saved: Cropped_Tumors\normal\normal_262.png
Saved: Cropped_Tumors\normal\normal_264.png


100%|██████████| 1355/1355 [00:01<00:00, 848.42it/s]

Processing complete. Cropped tumor images and normal images are saved in: Cropped_Tumors


1 random crops per normal image in order to increase the number of images we feed our CNN.  

In [11]:
import random

# Define output directory for cropped normal images
output_dir = "Cropped_Tumors"
normal_dir = os.path.join(output_dir, "normal")

os.makedirs(normal_dir, exist_ok=True)

# Random crop parameters
NUM_CROPS = 1  # Number of crops per normal image
CROP_SIZE = 128  # Size of each cropped region (128x128 pixels)

# Iterate through the DataFrame and process only normal ultrasound images
for index, row in tqdm(df.iterrows(), total=len(df)):
    image_path = row["image_path"]
    label = row["label"]  # 0 = normal

    # Skip non-normal images and all mask images
    if label != 0 or "_mask" in image_path:
        continue  

    # Load the normal ultrasound image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not load {image_path}")
        continue
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (1024, 1024))  # Resize for consistency

    # Generate random crops
    for crop_index in range(NUM_CROPS):
        # Randomly select a top-left corner for the crop (within image bounds)
        x = random.randint(0, 1024 - CROP_SIZE)
        y = random.randint(0, 1024 - CROP_SIZE)

        # Extract the crop
        crop = image[y:y + CROP_SIZE, x:x + CROP_SIZE]

        # Save the cropped image
        crop_filename = os.path.join(normal_dir, f"normal_{index}_crop{crop_index}.png")
        cv2.imwrite(crop_filename, cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))

        print(f"Saved: {crop_filename}")

print("Processing complete. Randomly cropped normal images are saved in:", normal_dir)


  2%|▏         | 31/1355 [00:00<00:08, 160.19it/s]

Saved: Cropped_Tumors\normal\normal_0_crop0.png
Saved: Cropped_Tumors\normal\normal_2_crop0.png
Saved: Cropped_Tumors\normal\normal_4_crop0.png
Saved: Cropped_Tumors\normal\normal_6_crop0.png
Saved: Cropped_Tumors\normal\normal_8_crop0.png
Saved: Cropped_Tumors\normal\normal_10_crop0.png
Saved: Cropped_Tumors\normal\normal_12_crop0.png
Saved: Cropped_Tumors\normal\normal_14_crop0.png
Saved: Cropped_Tumors\normal\normal_16_crop0.png
Saved: Cropped_Tumors\normal\normal_18_crop0.png
Saved: Cropped_Tumors\normal\normal_20_crop0.png
Saved: Cropped_Tumors\normal\normal_22_crop0.png
Saved: Cropped_Tumors\normal\normal_24_crop0.png
Saved: Cropped_Tumors\normal\normal_26_crop0.png
Saved: Cropped_Tumors\normal\normal_28_crop0.png
Saved: Cropped_Tumors\normal\normal_30_crop0.png
Saved: Cropped_Tumors\normal\normal_32_crop0.png
Saved: Cropped_Tumors\normal\normal_34_crop0.png
Saved: Cropped_Tumors\normal\normal_36_crop0.png
Saved: Cropped_Tumors\normal\normal_38_crop0.png
Saved: Cropped_Tumors\nor

  5%|▌         | 73/1355 [00:00<00:06, 190.09it/s]

Saved: Cropped_Tumors\normal\normal_42_crop0.png
Saved: Cropped_Tumors\normal\normal_44_crop0.png
Saved: Cropped_Tumors\normal\normal_46_crop0.png
Saved: Cropped_Tumors\normal\normal_48_crop0.png
Saved: Cropped_Tumors\normal\normal_50_crop0.png
Saved: Cropped_Tumors\normal\normal_52_crop0.png
Saved: Cropped_Tumors\normal\normal_54_crop0.png
Saved: Cropped_Tumors\normal\normal_56_crop0.png
Saved: Cropped_Tumors\normal\normal_58_crop0.png
Saved: Cropped_Tumors\normal\normal_60_crop0.png
Saved: Cropped_Tumors\normal\normal_62_crop0.png
Saved: Cropped_Tumors\normal\normal_64_crop0.png
Saved: Cropped_Tumors\normal\normal_66_crop0.png
Saved: Cropped_Tumors\normal\normal_68_crop0.png
Saved: Cropped_Tumors\normal\normal_70_crop0.png
Saved: Cropped_Tumors\normal\normal_72_crop0.png
Saved: Cropped_Tumors\normal\normal_74_crop0.png
Saved: Cropped_Tumors\normal\normal_76_crop0.png
Saved: Cropped_Tumors\normal\normal_78_crop0.png
Saved: Cropped_Tumors\normal\normal_80_crop0.png


  8%|▊         | 113/1355 [00:00<00:06, 187.88it/s]

...

Saved: Cropped_Tumors\normal\normal_174_crop0.png
Saved: Cropped_Tumors\normal\normal_176_crop0.png
Saved: Cropped_Tumors\normal\normal_178_crop0.png
Saved: Cropped_Tumors\normal\normal_180_crop0.png
Saved: Cropped_Tumors\normal\normal_182_crop0.png
Saved: Cropped_Tumors\normal\normal_184_crop0.png
Saved: Cropped_Tumors\normal\normal_186_crop0.png
Saved: Cropped_Tumors\normal\normal_188_crop0.png
Saved: Cropped_Tumors\normal\normal_190_crop0.png
Saved: Cropped_Tumors\normal\normal_192_crop0.png
Saved: Cropped_Tumors\normal\normal_194_crop0.png
Saved: Cropped_Tumors\normal\normal_196_crop0.png
Saved: Cropped_Tumors\normal\normal_198_crop0.png
Saved: Cropped_Tumors\normal\normal_200_crop0.png
Saved: Cropped_Tumors\normal\normal_202_crop0.png
Saved: Cropped_Tumors\normal\normal_204_crop0.png
Saved: Cropped_Tumors\normal\normal_206_crop0.png
Saved: Cropped_Tumors\normal\normal_208_crop0.png
Saved: Cropped_Tumors\normal\normal_210_crop0.png


 18%|█▊        | 247/1355 [00:01<00:05, 212.55it/s]

Saved: Cropped_Tumors\normal\normal_212_crop0.png
Saved: Cropped_Tumors\normal\normal_214_crop0.png
Saved: Cropped_Tumors\normal\normal_216_crop0.png
Saved: Cropped_Tumors\normal\normal_218_crop0.png
Saved: Cropped_Tumors\normal\normal_220_crop0.png
Saved: Cropped_Tumors\normal\normal_222_crop0.png
Saved: Cropped_Tumors\normal\normal_224_crop0.png
Saved: Cropped_Tumors\normal\normal_226_crop0.png
Saved: Cropped_Tumors\normal\normal_228_crop0.png
Saved: Cropped_Tumors\normal\normal_230_crop0.png
Saved: Cropped_Tumors\normal\normal_232_crop0.png
Saved: Cropped_Tumors\normal\normal_234_crop0.png
Saved: Cropped_Tumors\normal\normal_236_crop0.png
Saved: Cropped_Tumors\normal\normal_238_crop0.png
Saved: Cropped_Tumors\normal\normal_240_crop0.png
Saved: Cropped_Tumors\normal\normal_242_crop0.png
Saved: Cropped_Tumors\normal\normal_244_crop0.png
Saved: Cropped_Tumors\normal\normal_246_crop0.png
Saved: Cropped_Tumors\normal\normal_248_crop0.png
Saved: Cropped_Tumors\normal\normal_250_crop0.png


100%|██████████| 1355/1355 [00:01<00:00, 940.83it/s]

Saved: Cropped_Tumors\normal\normal_258_crop0.png
Saved: Cropped_Tumors\normal\normal_260_crop0.png
Saved: Cropped_Tumors\normal\normal_262_crop0.png
Saved: Cropped_Tumors\normal\normal_264_crop0.png
Processing complete. Randomly cropped normal images are saved in: Cropped_Tumors\normal


### Now we retrieve the index of the images of the validation set (that we will use to feed the CNN either through SAM or just by giving the full img)

In [12]:
import re

# Load the CSV file
file_path = "C:/Users/elisa/OneDrive/Desktop/DL_Final/validation_images.csv"  
df_index = pd.read_csv(file_path)

# Function to extract the index from the filename
def extract_index(filename):
    match = re.search(r"tumor_(\d+)_mask[0-2]\.png", filename)  
    if match:
        return int(match.group(1))
    return None  # Return None if no match is found

# Apply the function to extract the index
df_index["extracted_index"] = df_index["filename"].apply(extract_index)

df_index = df_index.drop(columns=['index', 'split'])
df_index

,filename,category,extracted_index
0,tumor_907_mask0.png,benign,907.0
1,tumor_331_mask0.png,benign,331.0
2,tumor_295_mask0.png,benign,295.0
3,tumor_883_mask0.png,benign,883.0
4,tumor_379_mask0.png,benign,379.0
...,...,...,...
161,normal_106_crop0.png,normal,NaN
162,normal_210.png,normal,NaN
163,normal_92.png,normal,NaN
164,normal_192.png,normal,NaN


In [13]:
#NA are from normal
df_filtered = df_index["extracted_index"]
df_filtered
df_filtered_NA = df_filtered.dropna()
df_filtered_NA

0       907.0
1       331.0
2       295.0
3       883.0
4       379.0
        ...  
107    1181.0
108    1165.0
109    1133.0
110    1059.0
111    1239.0
Name: extracted_index, Length: 112, dtype: float64

In [14]:
df_filtered_rows = df.loc[df_filtered_NA]
df_filtered_rows

,image_path,label
907,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,1
331,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,1
295,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,1
883,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,1
379,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,1
...,...,...
1181,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,2
1165,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,2
1133,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,2
1059,C:/Users/elisa/OneDrive/Desktop/DL_Final/Breas...,2


In [15]:
df_filtered_rows.to_csv("full_img_test.csv", index=True)
